<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/Cp2/checkpoint2_submission2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises: physics-informed neural network

Exercise on the implementation of physics-informed neural network.

Date: 2024

Course: 056936 - SCIENTIFIC COMPUTING TOOLS FOR ADVANCED MATHEMATICAL MODELLING (PAGANI STEFANO) [2023-24].

Example adapted from this [notebook](https://colab.research.google.com/drive/1qBrbgevkSBqqYc8bOPiaoJG1MBrBrluN?usp=share_link).


Let us consider the problem

\begin{aligned}
  & v_f *\sqrt(\nabla u\cdot D\nabla u) =1  \,, \quad (x,y) \in [-1.5,1.5] \times [-1.5,1.5]\,\\
\end{aligned}

where $\nu$ is unknown. We consider the PINN framework for solving the state/parameter estimation.

In [1]:
# import required libraries

import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import make_axes_locatable

!pip -q install pyDOE
from pyDOE import lhs

  Preparing metadata (setup.py) ... done


In [2]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp2

Cloning into 'ScientificTools'...
remote: Enumerating objects: 539, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 539 (delta 129), reused 226 (delta 68), pack-reused 156
Receiving objects: 100% (539/539), 129.20 MiB | 14.38 MiB/s, done.
Resolving deltas: 100% (167/167), done.
Updating files: 100% (24/24), done.
/content/ScientificTools/Project1/Cp2


In [8]:
# loading of the dataset

CP2data = np.load("CP2data.npz")
CP2data = CP2data['arr_0']

In [3]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [4]:
# collocation points
Ncl = 20000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

In [5]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))
# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,theta_fiber,a_ratio,y0):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = PINN(input_data)
    r_pred   = r_PINN(xcl,ycl,theta_fiber,a_ratio)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.abs(r_pred))

    # bc
    param_2= -1.5+(1.5+1.5)*y0
    mse_bc= tf.pow( PINN( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), param_2] ) ) ) ,2)

    #penalty over param boundaries
    mse_penalty = penalty(theta_fiber,0,1)+penalty(a_ratio,0,1)+penalty(y0,0,1)

    #tf.print('mse_time',mse_meas)
    #tf.print('mse_param',mse_r)
    #tf.print('mse_bc',mse_bc)
    return mse_meas + 2*mse_r + 2*mse_bc + mse_penalty


# residual computation based on AD
@tf.function
def r_PINN(x,y,theta_fiber,a_ratio):
    input_data=tf.concat([x,y],1)
    u = PINN(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    pi = tf.constant(np.pi,dtype=tf.float64)
    param_0=-np.pi/10+(np.pi/10+np.pi/10)*theta_fiber
    param_1=1+(9-1)*a_ratio
    theta0 = pi/2 - param_0
    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    #D = ((1/param[1])* tf.linalg.matmul(a,tf.transpose(a)) + tf.linalg.matmul(b,tf.transpose(b)))
    #return  tf.linalg.matmul(tf.transpose(u_grad), tf.linalg.matmul(D,u_grad)) - 1/100**2
    D_00 = 1 / param_1 * a[0]**2 + b[0]**2
    D_01 = 1 / param_1 * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / param_1 * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / param_1 * a[1]**2 + b[1]**2

    return ((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y))  - 1/100**2


# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,theta_fiber,a_ratio,y0):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,theta_fiber,a_ratio,y0)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grad_tf = tape.gradient(loss_value,theta_fiber)
        grad_ar = tape.gradient(loss_value,a_ratio)
        grad_y0 = tape.gradient(loss_value,y0)
    return loss_value, grads, grad_tf, grad_ar, grad_y0

In [6]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM


regularization_strength = 1e-3

PINN = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,),
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
     tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Reshape((1, 64)),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 64)),

    tf.keras.layers.Dense(128, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
     tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(32, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),
     tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(1, activation=None,
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64)
])


In [7]:
from scipy.interpolate import RBFInterpolator

def checkpoint1_solution(x, y, t, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5):
    coordinates = np.column_stack((x, y))

    mesh_coordinates=np.column_stack((X.ravel(), Y.ravel()))

    s = [s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_aniso_1, s_value,s_value,s_value,s_value, s_aniso_2,s_value,s_value,s_value,s_value]

    rbf = RBFInterpolator(coordinates, t, neighbors=None, smoothing=s, kernel='thin_plate_spline', epsilon=None, degree=1)

    time_pred = rbf(mesh_coordinates)
    time_pred=time_pred.reshape(1501,1501)

    return time_pred

In [9]:
def checkpoint2_solution( x, y , t ):

  xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(x, y, t, test_size=0.1)
  xmeas_train = tf.constant(xmeas_train.reshape(18, 1), dtype=tf.float64)
  ymeas_train = tf.constant(ymeas_train.reshape(18, 1), dtype=tf.float64)
  tmeas_train = tf.constant(tmeas_train.reshape(18, 1), dtype=tf.float64)
  xmeas_val = tf.constant(xmeas_val.reshape(2, 1), dtype=tf.float64)
  ymeas_val = tf.constant(ymeas_val.reshape(2, 1), dtype=tf.float64)
  tmeas_val = tf.constant(tmeas_val.reshape(2, 1), dtype=tf.float64)

  X, Y = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))
  time_pred = checkpoint1_solution(x, y, t, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5)
  y0_initial = Y[np.where(time_pred==np.min(time_pred))]
  y0_initial_scaled=(y0_initial+1.5)/3

  theta_fiber = tf.Variable([0.5], trainable=True,dtype=tf.float64)
  a_ratio = tf.Variable([0.5], trainable=True,dtype=tf.float64)
  y0 = tf.Variable([y0_initial_scaled[0]], trainable=True,dtype=tf.float64)


  pinn_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  pinn_optimizer.build(PINN.trainable_variables)

  param0_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  param0_optimizer.build([theta_fiber])

  param1_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  param1_optimizer.build([a_ratio])

  param2_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)
  param2_optimizer.build( [y0])


  for iter in range(800):

    loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

    pinn_optimizer.apply_gradients(zip(grads,PINN.trainable_variables))

    loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)

    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
      a_ratio_res=1+8*a_ratio
      y0_res=-1.5+3*y0
      print(theta_fiber_res.numpy())
      print(a_ratio_res.numpy())
      print(y0_res.numpy())


  for iter in range(800):


    loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

    param0_optimizer.apply_gradients(zip([grad_tf],[theta_fiber]))
    pinn_optimizer.apply_gradients(zip(grads,PINN.trainable_variables))

    loss_value_val, _, _, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
      a_ratio_res=1+8*a_ratio
      y0_res=-1.5+3*y0
      print(theta_fiber_res.numpy())
      print(a_ratio_res.numpy())
      print(y0_res.numpy())
      print()

  print()
  for iter in range(800):


    loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

    param1_optimizer.apply_gradients(zip([grad_ar],[a_ratio]))
    pinn_optimizer.apply_gradients(zip(grads,PINN.trainable_variables))

    loss_value_val, _, _, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
      a_ratio_res=1+8*a_ratio
      y0_res=-1.5+3*y0
      print(theta_fiber_res.numpy())
      print(a_ratio_res.numpy())
      print(y0_res.numpy())
      print()

  print()
  for iter in range(800):


    loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

    param2_optimizer.apply_gradients(zip([grad_y0],[y0]))
    pinn_optimizer.apply_gradients(zip(grads,PINN.trainable_variables))

    loss_value_val, _, _, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
      a_ratio_res=1+8*a_ratio
      y0_res=-1.5+3*y0
      print(theta_fiber_res.numpy())
      print(a_ratio_res.numpy())
      print(y0_res.numpy())
      print()
  """
  patience = float('inf')
  patience_lr= float('inf')
  min_delta = 1e-9
  best_val_loss = float('inf')
  wait = 0
  count = 0
  """
  pinn_optimizer.learning_rate=0.005
  param0_optimizer.learning_rate=0.003
  param1_optimizer.learning_rate=0.003
  param2_optimizer.learning_rate=0.003

  print()
  for iter in range(5000):


    loss_value,grads,grad_tf, grad_ar, grad_y0 = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, theta_fiber,a_ratio,y0)

    pinn_optimizer.apply_gradients(zip(grads,PINN.trainable_variables))
    param0_optimizer.apply_gradients(zip([grad_tf],[theta_fiber]))
    param1_optimizer.apply_gradients(zip([grad_ar],[a_ratio]))
    param2_optimizer.apply_gradients(zip([grad_y0],[y0]))


    loss_value_val, _, _,_,_ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val,theta_fiber,a_ratio,y0)

    """
    best_weigths = None
    best_params = None

    # Early stopping
    if loss_value_val < best_val_loss - min_delta:
        best_val_loss = loss_value_val
        wait = 0
        count = 0
        best_weights = PINN.get_weights()
        best_params = theta_fiber.numpy()
    else:
        wait += 1
        count += 1

        if count >= patience_lr:
          tf_optimizer.learning_rate = tf_optimizer.learning_rate * 0.9
          count = 0

        if wait >= patience:
            print('Early stopping at epoch', iter + 1)
            break
    """


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      theta_fiber_res= -np.pi/10+np.pi/5*theta_fiber
      a_ratio_res=1+8*a_ratio
      y0_res=-1.5+3*y0
      print(theta_fiber_res.numpy())
      print(a_ratio_res.numpy())
      print(y0_res.numpy())
      print()

  return theta_fiber_res, a_ratio_res, y0_res

In [ ]:
# measurement points
ind_disp = 0
xmeas = CP2data[ind_disp][0]
ymeas = CP2data[ind_disp][1]
tmeas = CP2data[ind_disp][2]
theta_fiber, a_ratio, y0 = checkpoint2_solution(xmeas, ymeas , tmeas)

iter =  100
loss = [[0.00024497835718349971]]
loss_val_param = [[0.00014578289150643506]]
[0.]
[5.]
[0.62]
iter =  200
loss = [[0.00015555257875025602]]
loss_val_param = [[0.00019553463209780094]]
[0.]
[5.]
[0.62]
iter =  300
loss = [[9.0691324912714629e-05]]
loss_val_param = [[6.2541922818241591e-05]]
[0.]
[5.]
[0.62]
iter =  400
loss = [[7.0769423000506312e-05]]
loss_val_param = [[4.7235452861437888e-05]]
[0.]
[5.]
[0.62]
iter =  500
loss = [[4.3145483246447505e-05]]
loss_val_param = [[2.985381360488135e-05]]
[0.]
[5.]
[0.62]
iter =  600
loss = [[3.77218365897134e-05]]
loss_val_param = [[2.6583535955221812e-05]]
[0.]
[5.]
[0.62]
iter =  700
loss = [[3.21651964924137e-05]]
loss_val_param = [[2.3995765162546069e-05]]
[0.]
[5.]
[0.62]
iter =  800
loss = [[2.9775083302401288e-05]]
loss_val_param = [[2.0197331520256153e-05]]
[0.]
[5.]
[0.62]
iter =  100
loss = [[2.7969771744638324e-05]]
loss_val_param = [[2.04690834600717e-05]]
[-0.05497025]
[5.]
[0.62]

iter =  200
loss = [[2.63909512443